# **Data3406 - Tpp9**

> Authur: Ran Sun

### **Notebook for further data cleanings and explorations.**



---
(This notebook is copied from the pre-processed dataset from tpp08.)




> *Importing libraries required for data cleanings and explorations below.*







In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


> Reading modified data from a csv file.





In [4]:
M5=pd.read_csv('/content/drive/MyDrive/Modifydata/modifydata.csv')

In [5]:
M5.head() # Check the structure of dataset.
print(M5.dtypes)# Check type of variables.

Date     object
Count     int64
dtype: object




> Convert object type into datetime types


In [6]:
M5['Date'].replace('-','',regex=True,inplace=True) # replace some unwanted parts of character.
M5.head()
M5['Date'] = pd.to_datetime(M5['Date'], format='%Y%m%d')# convert it into normalized forms.
print(M5.dtypes)

Date     datetime64[ns]
Count             int64
dtype: object




> Add a new column to convert date into week number.




In [7]:
M5['weekcount'] = M5['Date'].dt.isocalendar().week
M5.head()

,Date,Count,weekcount
0,2022-01-01,114,52
1,2022-01-02,8681,52
2,2022-01-03,5104,1
3,2022-01-04,8031,1
4,2022-01-05,7743,1




> To keep data consistencyt,drop first two rows cuz they are in week 52 and start to analyze the data from week1 to the end of this dateset.



In [8]:
M5 = M5.iloc[2: , :] # drop two rows which is useless in this case.
M5

,Date,Count,weekcount
2,2022-01-03,5104,1
3,2022-01-04,8031,1
4,2022-01-05,7743,1
5,2022-01-06,8948,1
6,2022-01-07,2707,1
...,...,...,...
217,2022-08-28,12888,34
218,2022-08-29,6036,35
219,2022-08-30,9820,35
220,2022-08-31,10523,35




> Next, calculate the sum of counts in each week and updating the weekcount names.



In [9]:
M5['weekcount']=M5['weekcount'].astype(str) #convert it into object type
groupM5=M5.groupby('weekcount',sort=False)['weekcount','Count'].sum() # make it starts with week1
groupM5
from google.colab import files
groupM5.to_csv('modifydata_update.csv', sep=',')
files.download('modifydata_update.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
M5update=pd.read_csv('/content/drive/MyDrive/Modifydata/modifydata_update.csv')
M5update.head()

,weekcount,Count
0,1,32533
1,2,347
2,4,4
3,5,24943
4,6,48152


In [12]:
M5update['weekcount']=M5update['weekcount'].astype(str)
for i in range(len(M5update)):
  M5update['weekcount'].values[i] = 'week'+M5update['weekcount'].values[i] # iterate every row and update its value
M5update.head()

,weekcount,Count
0,week1,32533
1,week2,347
2,week4,4
3,week5,24943
4,week6,48152


According to the [essay](https://link.springer.com/article/10.2165/00007256-200434010-00001), 10000 steps/day appears to be a reasonable estimate of daily activity for adults to be healthy.
Hence, using 1(`healthy level`) and 0（`healthy level not reached`）to reprsent weather this participant is on the healthy track for each week.

In [13]:
standard=70000
M5update['determine']=''
for i in range(len(M5update)):
  if M5update['Count'].values[i]>standard:
    M5update['determine'].values[i]='1'
  else:
     M5update['determine'].values[i]='0'
M5update.head()

,weekcount,Count,determine
0,week1,32533,0
1,week2,347,0
2,week4,4,0
3,week5,24943,0
4,week6,48152,0




> Now, using logistic regression  to predict the probability of staying on healthy track. [reference](https://towardsdatascience.com/building-a-logistic-regression-in-python-step-by-step-becd4d56c9c8)


In [14]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

In [25]:
M5update['determine']=M5update['determine'].astype(int) # keep data type consistent
X= M5update['Count'].values
Y= M5update['determine'].values
import statsmodels.api as sm
logit_model=sm.Logit(Y,X)
result=logit_model.fit()
print(result.summary2())
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

Optimization terminated successfully.
         Current function value: 0.693145
         Iterations 2
                        Results: Logit
Model:              Logit            Pseudo R-squared: -0.101 
Dependent Variable: y                AIC:              49.1339
Date:               2022-10-02 17:15 BIC:              50.6602
No. Observations:   34               Log-Likelihood:   -23.567
Df Model:           0                LL-Null:          -21.403
Df Residuals:       33               LLR p-value:      nan    
Converged:          1.0000           Scale:            1.0000 
No. Iterations:     2.0000                                    
-----------------------------------------------------------------
      Coef.     Std.Err.      z       P>|z|      [0.025    0.975]
-----------------------------------------------------------------
x1    0.0000      0.0000    0.0123    0.9902    -0.0000    0.0000

